# Demand prediction baseline solution

Victor Kantor, xead.wl@gmail.com

(a) Задача: спрогнозировать спрос на 3 недели в 2015 год по данным предыдущих годов

(b)(c) Данные получали из файла train.tsv. Из выборки выделяли target - 'y',  В качестве признаков использовали все столбцы кроме 'Num' и 'y'. При решении задачи использовали градиентный бустинг. Меняли параметры - глубина дерева, вклад, который вносит каждый алгоритм, количество деревьев. При увеличении глубины дерева наступало переобучение. Чтобы избавиться от переобучения, увеличивали количество дерьевьев и  уменьшали вклад, который дает каждый алгоритм. При этом алгоритм работал относительно долго, но качество получалось лучше, чем при, например, когда каждый алгоритм давал полный вклад (т.е коэффициент = 1) в общий алгоритм. 

(e) Для того чтобы оценивать качество исходную выборку train делили на обучающуюся и тестируемую с помощью кросс валлидации. И пользовались функцией SMAPE. 

In [33]:
import pandas as pd
import numpy as np

train = pd.read_csv("train.tsv")
test = pd.read_csv("test.tsv")
sample_submission = pd.read_csv("sample_submission.tsv")

In [34]:
frac = 0.5 # fraction of learning examples used for model fitting

#sample the train set if your don't want to deel with all examples
train = train.sample(frac=frac, random_state=47)

X = train.drop(['Num','y'], axis=1)
y = train['y']

SMAPE Для оценки точности предсказания

In [35]:
def smape(pred, y):
        return float(200) / len(y) * sum(abs(pred - y) / (abs(y) + abs(pred)))

Для тестирования алгоритма разбиваем выборку на обучающуюся и на тестовую

In [36]:
from sklearn import cross_validation

X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.20, random_state=1)

Используем градиентный бустинг. Подбор параметров осуществлялся так: при увеличении высоты дерева качество улучшалось. Чтобы не было переобучения, увеличивали количество деревьев и уменьшали вклад, который вносит каждый алгоритм. Это увеличивает время работы алгоритма, но улучшает точность.

In [37]:
%%time

import xgboost as xgb

model = xgb.XGBRegressor(max_depth=20, learning_rate=0.1, n_estimators=1000)

model.fit(X_train, y_train)

# max_depth = 20 better

CPU times: user 24min 26s, sys: 3.11 s, total: 24min 29s
Wall time: 6min 22s


In [38]:
preds = model.predict(X_test)

На тестовой и обучающей выборке получался такой результат

In [39]:
smape(preds, np.array(y_test))

23.057478013680193

Теперь всю выборку train возьмем за обучающуюся (точнее ее часть - frac).

In [40]:
%%time
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb

model = xgb.XGBRegressor(max_depth=25, learning_rate=0.1, n_estimators=1000)


model.fit(X, y)
preds = model.predict(test.drop(['Num'], axis=1))


CPU times: user 36min 16s, sys: 2.47 s, total: 36min 18s
Wall time: 9min 9s


In [41]:
sample_submission['y'] = preds

In [42]:
sample_submission.head(5)

,Num,y
0,348622,1281.580688
1,348623,25850.800781
2,348624,306773.781250
3,348625,30670.210938
4,348626,233.975540


In [43]:
# In GBM you can get some negative predictions:
print sample_submission[sample_submission['y'] < 0]

Empty DataFrame
Columns: [Num, y]
Index: []


In [44]:
sample_submission['y'] = sample_submission['y'].map(lambda x: x if x > 0 else 0.0)

In [45]:
sample_submission.to_csv("baseline_submission.tsv", sep=',', index=False)